In [1]:
#Namen der Inseln

Inseln = [
    "Cuarteron_Reef",
    "Fiery_Cross_Reef",
    "Gaven_Reefs",
    "Hughes_Reef",
    "Johnson_Reef",
    "Mischief_Reef",
    "Subi_Reef",
    "Antelope_Reef",
    "Drummond_Island",
    "Duncan_Islands",
    "Lincoln_Islands",
    "Middle_Island",
    "Money_Island",
    "North_Island",
    "North_Reef",
    "Observation_Bank",
    "Pattle_Island",
    "Quanfu_Island",
    "Robert_Island",
    "South_Island",
    "South_Sand",
    "Tree_Island",
    "Triton_Island",
    "West_Sand",
    "Woody_Island",
    "Yagong_Island",
    "Scarborough_Shoal",
    "Bombay_Reef",
    "Ardasier_Reef",
    "Erica_Reef",
    "Investigator_Shoal",
    "Mariveles_Reef",
    "Swallow_Reef",
    "Commodore_Reef",
    "Flat_Island",
    "Loaita_Cay",
    "Loaita_Island",
    "Nanshan_Island",
    "Northeast_Cay",
    "Second_Thomas_Shoal",
    "Thitu_Island",
    "West_York_Island",
    "Itu_Aba_Island",
    "Alison_Reef",
    "Amboyna_Cay",
    "Barque_Canada_Reef",
    "Central_Reef",
    "Collins_Reef",
    "Cornwallis_South_Reef",
    "Discovery_Great_Reef",
    "East_Reef",
    "Grierson_Reef",
    "Ladd_Reef",
    "Lansdowne_Reef",
    "Namyit_Island",
    "Pearson_Reef",
    "Petley_Reef",
    "Sand_Cay",
    "Sin_Cowe_Island",
    "South_Reef",
    "Southwest_Cay",
    "Spratly_Island",
    "Tennent_Reef",
    "West_Reef",
    "Alexandra_Bank",
    "Grainger_Bank",
    "Prince_Consort_Bank",
    "Prince_of_Wales_Bank",
    "Rifleman_Bank",
    "Vanguard_Bank"
]

In [3]:
import os
import numpy as np
import rasterio
import pandas as pd #für .csv-Tabelle


#Funktion zur Berechnung des NDWI erstellen
def compute_ndwi(tiff_path, green_band=1, nir_band=2):
    with rasterio.open(tiff_path) as src:
        green = src.read(green_band).astype(np.float32)
        nir = src.read(nir_band).astype(np.float32)

        pixel_size = src.res[0]
        pixel_area_m2 = pixel_size ** 2

    with np.errstate(divide='ignore', invalid='ignore'):
        ndwi = (green - nir) / (green + nir)
    ndwi[np.isinf(ndwi)] = np.nan

    return ndwi, pixel_area_m2

#Funktion, um die jeweiligen vorher/nachher-TIFFs ins Script zu laden
def get_images_for_insel(insel):
    base = r"C:\Users\Windows\Desktop\Master\GIS1\TIFFs"
    s2_1 = os.path.join(base, "S2_1", f"S2_1_{insel}.tif")
    s2_2 = os.path.join(base, "S2_2", f"S2_2_{insel}.tif")

    if not os.path.exists(s2_1):
        raise FileNotFoundError(s2_1)
    if not os.path.exists(s2_2):
        raise FileNotFoundError(s2_2)

    ndwi_1, pixel_area = compute_ndwi(s2_1)
    ndwi_2, _ = compute_ndwi(s2_2)

    return ndwi_1, ndwi_2, pixel_area


In [4]:
results = []

#For-Schleife, die für alle Inseln anzeigt, wie viele Pixel Land waren/sind (in m²):
for insel in Inseln:
    ndwi_1, ndwi_2, pixel_area = get_images_for_insel(insel)

    land_1 = np.where(np.isnan(ndwi_1), 0, ndwi_1 <= 0).astype(np.uint8)
    land_2 = np.where(np.isnan(ndwi_2), 0, ndwi_2 <= 0).astype(np.uint8)

    land_pixels_1 = np.sum(land_1)
    land_pixels_2 = np.sum(land_2)

    land_area_1 = land_pixels_1 * pixel_area
    land_area_2 = land_pixels_2 * pixel_area

    print(f"{insel}: Landfläche vorher (S2_1) = {land_area_1:.1f} m², nachher (S2_2) = {land_area_2:.1f} m²")

    results.append({
        "insel": insel,
        "land_1": land_1,
        "land_2": land_2,
        "land_area_1_m2": land_area_1,
        "land_area_2_m2": land_area_2
    })


Cuarteron_Reef: Landfläche vorher (S2_1) = 168600.0 m², nachher (S2_2) = 597000.0 m²
Fiery_Cross_Reef: Landfläche vorher (S2_1) = 500400.0 m², nachher (S2_2) = 3139300.0 m²
Gaven_Reefs: Landfläche vorher (S2_1) = 200700.0 m², nachher (S2_2) = 132800.0 m²
Hughes_Reef: Landfläche vorher (S2_1) = 154200.0 m², nachher (S2_2) = 81300.0 m²
Johnson_Reef: Landfläche vorher (S2_1) = 1846500.0 m², nachher (S2_2) = 110300.0 m²
Mischief_Reef: Landfläche vorher (S2_1) = 5826000.0 m², nachher (S2_2) = 6165500.0 m²
Subi_Reef: Landfläche vorher (S2_1) = 258500.0 m², nachher (S2_2) = 298100.0 m²
Antelope_Reef: Landfläche vorher (S2_1) = 385000.0 m², nachher (S2_2) = 38900.0 m²
Drummond_Island: Landfläche vorher (S2_1) = 608100.0 m², nachher (S2_2) = 221000.0 m²
Duncan_Islands: Landfläche vorher (S2_1) = 748100.0 m², nachher (S2_2) = 734800.0 m²
Lincoln_Islands: Landfläche vorher (S2_1) = 1701300.0 m², nachher (S2_2) = 1852600.0 m²
Middle_Island: Landfläche vorher (S2_1) = 134500.0 m², nachher (S2_2) = 

In [5]:
#For-Schleife, die final ausrechnet wie viele m² dazugewonnen wurden:
for island_result in results:
    insel = island_result["insel"]
    area_1 = island_result["land_area_1_m2"]
    area_2 = island_result["land_area_2_m2"]

    diff = area_2 - area_1
    
    if diff > 0:
        status = "Landgewinn"
    elif diff < 0:
        status = "Verlust (Wolken?)"
    else:
        status = "unverändert"

    print(
        f"{insel}: {status} "
        f"({diff:+.0f} m²)" #.0 = keine Nachkommastellen
    )

Cuarteron_Reef: Landgewinn (+428400 m²)
Fiery_Cross_Reef: Landgewinn (+2638900 m²)
Gaven_Reefs: Verlust (Wolken?) (-67900 m²)
Hughes_Reef: Verlust (Wolken?) (-72900 m²)
Johnson_Reef: Verlust (Wolken?) (-1736200 m²)
Mischief_Reef: Landgewinn (+339500 m²)
Subi_Reef: Landgewinn (+39600 m²)
Antelope_Reef: Verlust (Wolken?) (-346100 m²)
Drummond_Island: Verlust (Wolken?) (-387100 m²)
Duncan_Islands: Verlust (Wolken?) (-13300 m²)
Lincoln_Islands: Landgewinn (+151300 m²)
Middle_Island: Verlust (Wolken?) (-3500 m²)
Money_Island: Verlust (Wolken?) (-608400 m²)
North_Island: Landgewinn (+361300 m²)
North_Reef: Verlust (Wolken?) (-1672400 m²)
Observation_Bank: Verlust (Wolken?) (-141800 m²)
Pattle_Island: Landgewinn (+58800 m²)
Quanfu_Island: Verlust (Wolken?) (-89600 m²)
Robert_Island: Verlust (Wolken?) (-23000 m²)
South_Island: Landgewinn (+5300 m²)
South_Sand: Verlust (Wolken?) (-300 m²)
Tree_Island: Verlust (Wolken?) (-93100 m²)
Triton_Island: Landgewinn (+1282400 m²)
West_Sand: Verlust (Wolk

In [ ]:
#.csv-Tabelle mit Ergebnissen erstellen lassen

rows = []

for result in results:
    land_2018 = result["land_area_1_m2"]
    land_2025 = result["land_area_2_m2"]
    diff = land_2025 - land_2018

    rows.append({
        "Name": result["insel"],
        "Landfläche 2018 [m²]": land_2018,
        "Landfläche 2025 [m²]": land_2025,
        "Differenz der Landflächen [m²]": diff
    })

df = pd.DataFrame(rows)

csv_path = r"C:\Users\Windows\Desktop\Master\GIS1/Landgewinn_Daten.csv"
df.to_csv(csv_path, index=False, encoding="utf-8-sig") #utf-8-sig kümmert sich um Sonderzeichen wie ä und ²

print(f"Tabelle gespeichert unter: {csv_path}")

Tabelle gespeichert unter: C:\Users\Windows\Desktop\Master\GIS1/Landgewinn_Daten.csv
